<br><br><br><br><br><br><br><br>
# window functions in postgres: they can show you things
<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>
<br><br><br><br><br><br><br><br>

<img src="http://blob.devolutions.net/images/Sysadminotaur/Live-Demo-Remote-Desktop-Manager-Maurice.png">

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
db_engine = create_engine('postgres://localhost:5432/funzies')

RAW_TABLE_NAME = 'v0_cannabis_prices'
VIEW_NAME = 'cannabis_prices_view'
CSV_URL='http://blog.yhathq.com/static/misc/data/marijuana-street-price-clean.csv'

In [ ]:
prices_df = pd.read_csv(CSV_URL, parse_dates=[7])
prices_df.drop(prices_df.columns[[2,3,4,5,6]], axis=1, inplace=True)
prices_df.columns = ['state', 'high_quality_price', 'recorded_at']

prices_df.to_sql(RAW_TABLE_NAME, db_engine, if_exists='replace', index=False)

In [ ]:
def show_me_some_data(query, limit=10):
    df = pd.read_sql(query, db_engine)
    return df.head(limit)


def create_view(query, view_name=VIEW_NAME):
    create_view_query = """
        DROP MATERIALIZED VIEW IF EXISTS %(view_name)s; 
        CREATE MATERIALIZED VIEW %(view_name)s AS %(query)s;""" % {'view_name': view_name, 'query': query}
    db_engine.connect().execute(create_view_query)

    
def show_me_whats_in_the_view(view_name=VIEW_NAME, limit=10):
    return show_me_some_data("select * from %(view_name)s" % {'view_name': view_name}, limit)
    

## see it in action




In [ ]:
create_view("""
    SELECT state
        , recorded_at
        , high_quality_price
    FROM %(RAW_TABLE_NAME)s 
    WHERE state in ('New York', 'New Jersey', 'Pennsylvania', 'Delaware', 'Colorado')
    AND recorded_at >= '2015-06-07'
    ORDER BY state
""" % locals())


show_me_whats_in_the_view(limit=40)

<br><br><br><br><br><br><br><br><br><br><br><br>
# cool, so window functions!

<br><br><br><br><br><br><br><br><br><br><br><br>


# ranking things

In [ ]:
create_view("""
    SELECT state
        , recorded_at
        , high_quality_price
        
        , ROW_NUMBER() OVER (
            PARTITION BY state ORDER BY high_quality_price
            ) rank_of_cheapest_day_in_the_state
        
        , ROW_NUMBER() OVER (
            PARTITION BY recorded_at ORDER BY high_quality_price
            ) rank_of_cheapest_state_each_day

    FROM %(RAW_TABLE_NAME)s 
    WHERE state in ('New York', 'New Jersey', 'Pennsylvania', 'Delaware', 'Colorado')
    AND recorded_at >= '2015-06-07'
    ORDER BY state
""" % locals())


show_me_whats_in_the_view(limit=40)

# comparing each row with the min and the argmin

In [ ]:
create_view("""
    SELECT state
        , recorded_at
        , high_quality_price


        , FIRST_VALUE(high_quality_price) OVER (
            PARTITION BY state ORDER BY high_quality_price
            ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
          ) cheapest_price_in_the_state


        , FIRST_VALUE(recorded_at) OVER (
            PARTITION BY state ORDER BY high_quality_price
            ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
          ) date_of_cheapest_price_in_the_state



    FROM %(RAW_TABLE_NAME)s 
    WHERE state in ('New York', 'New Jersey')
    AND recorded_at >= '2015-06-01'
    ORDER BY state
""" % locals())


show_me_whats_in_the_view(limit=40)



# yesterday and tomorrow

In [ ]:
create_view("""
    SELECT state
        , recorded_at
        , high_quality_price


        , LAG(high_quality_price) OVER (
            PARTITION BY state ORDER BY recorded_at
          ) price_yesterday


        , LEAD(high_quality_price) OVER (
            PARTITION BY state ORDER BY recorded_at
          ) price_tomorrow


    FROM %(RAW_TABLE_NAME)s 
    WHERE state in ('New York', 'New Jersey')
    AND recorded_at >= '2015-06-01'
    ORDER BY state
""" % locals())


show_me_whats_in_the_view(limit=40)



# one of my faves: ntile

In [ ]:

# let's scope down to just one state and add an extra month of prices

create_view("""
    SELECT state
        , recorded_at
        , high_quality_price


        , NTILE(4) OVER w price_date_quartile
        , NTILE(10) OVER w price_date_decile


    FROM %(RAW_TABLE_NAME)s 
    WHERE state in ('New York')
    AND recorded_at >= '2015-05-01'
    WINDOW w AS (PARTITION BY state ORDER BY high_quality_price)
    ORDER BY recorded_at
""" % locals())


show_me_whats_in_the_view(limit=40)


# window functions vs group-by / distinct-on

In [ ]:
# group by for cheapest price in the state

create_view("""
    SELECT state
    
        , MIN(high_quality_price) cheapest_price_in_the_state
    
    FROM %(RAW_TABLE_NAME)s 
    WHERE state in ('New York', 'New Jersey')
    AND recorded_at >= '2015-06-01'
    GROUP BY state
    ORDER BY state
""" % locals())

show_me_whats_in_the_view(limit=40)




In [ ]:
# distinct on for cheapest price in the state AND the arg min date

create_view("""
    SELECT distinct on (state) state
        , high_quality_price as cheapest_price_in_the_state
        , recorded_at as date_of_cheapest_price
    FROM %(RAW_TABLE_NAME)s 
    WHERE state in ('New York', 'New Jersey')
    AND recorded_at >= '2015-06-01'
    ORDER BY state, high_quality_price asc
""" % locals())
show_me_whats_in_the_view(limit=40)



# cardinality:
# the number of elements in a set or other grouping, as a property of that grouping.

<img src='http://support.prometheanplanet.com/upload/img_400/Counting-375x294.jpg'>


<h1>list of all available window functions in postgres

<h2>
* row_number()
* rank()
* dense_rank()
* percent_rank()
* cume_dist()
* ntile(num_buckets integer)
* lag(value)
* lead(value)
* first_value(value)
* last_value(value)
* nth_value(value, nth integer)

<h3> if you're really smart, you can probably write your own. other people have done it. so what are you waiting for?


<br><br><br>
# when to consider window functions
-------------------------------------------------------------
<h1>
* you want to enrich data, not necessarily answer a question
* particularly powerful with time series data
* you don't have access to `:distinct on:` but still want to do ARGMIN behavior (e.g. redshift)
* ranking
* filling in missing data with previous or future numbers
<br><br><br>
<br><br><br>
